In [ ]:
from models.models import *
from utils.datasets import *
import numpy as np

In [ ]:
# Load model
cfg = './cfg/yolor_p6.cfg'
imgsz = 640
weight = "./weights/best.pt"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

model = Darknet(cfg, imgsz).cuda()
model.load_state_dict(torch.load(weight, map_location=device)['model'])
#model = attempt_load(weights, map_location=device)  # load FP32 model
#imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
model.to(device).eval()
half = device.type != 'cpu' 
if half:
    print("HALF")
    model.half()  # to FP16

In [ ]:
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once

In [ ]:
names = [
        'headline',
        'doc',
        'cir_stamp',
        'rec_stamp'
]

In [ ]:
def relocate_illigal_point(point, pmin, pmax):
    point = round(point)
    if point < pmin:
        return pmin
    if point > pmax:
        return pmax
    return point

In [ ]:
def single_inference(img_path, model, conf_thres = 0.4, iou_thres=0.5):
    with torch.no_grad():
        # dataset = LoadImages(source, img_size=imgsz, auto_size=64)
        ori_img = cv2.imread(img_path)
        ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
        img_h, img_w, c = ori_img.shape
        print(ori_img.shape)
        img = letterbox(ori_img, new_shape=imgsz, auto_size=64)[0]
        # img = letterbox(img0, new_shape=self.img_size, auto_size=self.auto_size)[0]

                # Convert
        scaled_h, scaled_w, c = img.shape
        h_ratio = scaled_h/img_h
        w_ratio = scaled_w/img_w
        
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)


        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        # t1 = time_synchronized()
        pred = model(img)[0]
        pred = non_max_suppression(pred, conf_thres=conf_thres, iou_thres=0.5, classes=None) #[[left, top, right, bottom, probs, cls]]
        np_pred = pred[0].cpu().detach().numpy()

        draw_img = ori_img.copy()
        for each_pred in np_pred:
            relocate_point = []
            left, top, right, bottom, probs, cls = each_pred
            left = left/w_ratio
            right = right/w_ratio
            top = top/h_ratio
            bottom = bottom/h_ratio

            left = relocate_illigal_point(left, 0, img_w)
            top = relocate_illigal_point(top, 0, img_h)
            right = relocate_illigal_point(right, 0, img_w)
            bottom = relocate_illigal_point(bottom, 0, img_h)

            cv2.rectangle(draw_img, (left, top), (right, bottom), (255,0,0), 1)
    return draw_img

In [ ]:
img_path = "01F52F8C-1EA0-4DC4-821B-068FF7D30C00_IzCfN_1670037805225.jpg"
draw_img = single_inference(img_path, model, conf_thres=0.5, iou_thres=0.1)

In [ ]:
display(Image.fromarray(draw_img))

In [ ]:
from PIL import Image
from IPython.display import display

In [ ]:
for i, det in enumerate(pred):
    p, s, im0 = img_path, '', img
    s += '%gx%g ' % img.shape[2:]  # print string
    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
    if det is not None and len(det):
        # Rescale boxes from img_size to im0 size
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

        # Print results
        for c in det[:, -1].unique():
            n = (det[:, -1] == c).sum()  # detections per class
            s += '%g %ss, ' % (n, names[int(c)])  # add to string

In [ ]:
def single_infer(img_path):
    # t0 = time.time()
    source = "./output.png"
    dataset = LoadImages(source, img_size=imgsz, auto_size=64)
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        # t1 = time_synchronized()
        pred = model(img)[0]
        pred = non_max_suppression(pred, conf_thres=0.4, iou_thres=0.5, classes=None)


In [ ]:
a = 3.4e+17
'{:17f}'.format(a)

In [ ]:
len("340000000000000000")

In [1]:
from inference import DocumentLayoutDetection
import cv2

In [2]:
cfg_path = "./custom_cfg/ppy_yolor_p6_4cls.cfg"
weight_path = "./weights/best.pt"
DocLayoutDet = DocumentLayoutDetection(cfg_path, weight_path, device="cpu")

Document detection model loading completed by cpu


In [ ]:
test_img = cv2.imread("a.jpg")
detected_objs = DocLayoutDet(test_img)

In [4]:
detected_objs

defaultdict(list,
            {0: [{'bbox': [273, 202, 490, 230],
               'score': 0.8559931,
               'cls_name': 'headline',
               'cls_id': 0}],
             1: [{'bbox': [0, 84, 798, 662],
               'score': 0.79706466,
               'cls_name': 'doc',
               'cls_id': 1}],
             2: [{'bbox': [102, 179, 237, 307],
               'score': 0.746423,
               'cls_name': 'cir_stamp',
               'cls_id': 2}]})

In [5]:
bbox_color = [
    (252, 200, 154),
    (255, 248, 230),
    (255, 148, 172),
    (80, 92, 153),
]

In [ ]:
draw_img = test_img.copy()
for each_cls, each_objs in detected_objs.items:
    for each_obj in each_objs:
        left, top, right, bottom = each_obj["bbox"]
